In [6]:
from collections import Counter
from flask import Flask, request, render_template, jsonify
import pickle
import pandas as pd
from matplotlib import pyplot as plt
import numpy as np
import requests
import re
from bs4 import BeautifulSoup
import argparse
import time
from splinter import Browser
import time
from collections import defaultdict
import json
import re

In [22]:
import uuid

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
from src.cleaner import Cleaner 
import src.fantasy_scoring as fs

In [4]:
import requests
import boto3
from boto3.dynamodb.conditions import Key, Attr
import json
from decimal import Decimal

In [97]:
#session = requests.Session()
session = boto3.Session(profile_name='personal')

right = old

In [17]:
session.profile_name

'personal'

In [15]:
dynamodb = boto3.resource('dynamodb', region_name='us-east-1',aws_access_key_id = '',
aws_secret_access_key = '')
table = dynamodb.Table('nba_bbref')

In [98]:
dynamodb = session.client('dynamodb', region_name='us-east-1')

In [24]:
%timeit [uuid.uuid4() for i in range(100)]

330 µs ± 6.87 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [8]:
data1 = pd.read_csv("data/bb_reference/2016to2019Data_merged.csv", encoding = 'latin-1')
data2 = pd.read_csv("data/bb_reference/2013to2015Data_merged.csv", encoding = 'latin-1')

In [10]:
print(data1.shape)
print(data2.shape)
data = data1.append(data2)
data.shape

(101014, 32)
(83567, 32)


(184581, 32)

In [11]:
data = data.drop(['Unnamed: 0_y','Unnamed: 0'], axis = 1)
data = data.rename(columns={'Unnamed: 0_x':'gamescore_rank'})

In [12]:
clnr = Cleaner()
data = clnr.transform(data)

In [13]:
data['date']=data['dt'].apply(lambda x: x.strftime("%Y-%m-%d"))

In [99]:
table = dynamodb.Table('nba_bbref')  


AttributeError: 'DynamoDB' object has no attribute 'Table'

In [84]:
dynamodb.batch_write_item()

In [16]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 184581 entries, 0 to 21889
Data columns (total 54 columns):
gamescore_rank                       184581 non-null int64
assists                              184581 non-null int64
attempted_field_goals                184581 non-null int64
attempted_free_throws                184581 non-null int64
attempted_three_point_field_goals    184581 non-null int64
blocks                               184581 non-null int64
date                                 184581 non-null object
defensive_rebounds                   184581 non-null int64
game_score                           184581 non-null float64
location                             184581 non-null object
made_field_goals                     184581 non-null int64
made_free_throws                     184581 non-null int64
made_three_point_field_goals         184581 non-null int64
name                                 184581 non-null object
offensive_rebounds                   184581 non-null int64


In [20]:
players = data['slug'].unique()
names = data['player'].unique()
slug_names = {player:name for player,name in zip(players,names)}
name_slugs = {name:player for player,name in zip(players,names)}
type(players)

numpy.ndarray

In [22]:
players[0:20]

array(['horfoal01', 'aminual01', 'couside01', 'turneev01', 'griffbl01',
       'outlatr01', 'garcifr01', 'templga01', 'chandty01', 'bynumwi01',
       'batumni01', 'tollian01', 'martica01', 'crawfjo02', 'anthoca01',
       'koufoko01', 'brownkw01', 'maxieja01', 'artesro01', 'mcgeeja01'],
      dtype=object)

In [178]:
single = data[data['player']=='John Wall']
lag = clnr.gameplay_stats(single,player_col = 'slug',lag = 10)
lag['id']= lag['date']+lag['slug']
for ind, elem in enumerate(lag.iloc[0,:]):
    if (str(type(elem))=="<class 'numpy.float64'>"):
        lag.iloc[:,ind] = lag.iloc[:,ind].apply(lambda x : Decimal(x).quantize(Decimal('.001')))

In [21]:
name_slugs['John Wall']

'walljo01'

In [29]:
len(players)

1035

In [ ]:
lags = [1,2,3,6,11,22,33]
for player in players[20:30]:
    now = time.time()
    for lag in lags:
        single = data[data['slug']==player]
        lag_df = clnr.gameplay_stats(single,player_col = 'slug',lag = lag)
        if len(lag_df)>0:
            lag_df['id']= lag_df['date']+lag_df['slug']
            lag_df['name']=slug_names[player]
            for ind, elem in enumerate(lag_df.iloc[0,:]):
                if (str(type(elem))=="<class 'numpy.float64'>"):
                    lag_df.iloc[:,ind] = lag_df.iloc[:,ind].apply(lambda x : Decimal(x).quantize(Decimal('.001')))
            dict_row = lag_df.iloc[0,:].to_dict()
            key  = dict_row.pop('id')
            ean = {"#"+key : key for key in dict_row.keys()}
            ue = "SET " + ', '.join(["#"+key +" = :"+key for key in dict_row.keys()])
            with table.batch_writer() as batch:
                for row in lag_df.iterrows():
                    dict_row = row[1].to_dict()
                    key  = dict_row.pop('id')
                    eav = {":"+key : dict_row[key] for key in dict_row.keys()}
                    table.update_item(Key={'id':key},
                             ExpressionAttributeNames = ean,
                             ExpressionAttributeValues = eav,
                             UpdateExpression = ue)
        print("done with {} lag ({})".format(lag,len(lag_df)))
    print("Done with {} in {:.01f}".format(slug_names[player],(time.time()-now)/60))
            

done with 1 lag (282)
done with 2 lag (282)
done with 3 lag (282)
done with 6 lag (282)
done with 11 lag (282)
done with 22 lag (282)
done with 33 lag (282)
Done with Tayshaun Prince in 6.5
done with 1 lag (543)
done with 2 lag (543)
done with 3 lag (543)
done with 6 lag (543)
done with 11 lag (543)
done with 22 lag (543)
done with 33 lag (543)
Done with LaMarcus Aldridge in 25.3
done with 1 lag (201)
done with 2 lag (201)
done with 3 lag (201)
done with 6 lag (201)
done with 11 lag (201)
done with 22 lag (201)
done with 33 lag (201)
Done with Charlie Villanueva in 9.4
done with 1 lag (356)
done with 2 lag (356)
done with 3 lag (356)
done with 6 lag (356)
done with 11 lag (356)
done with 22 lag (356)
done with 33 lag (356)
Done with Nick Young in 16.6
done with 1 lag (172)
done with 2 lag (172)
done with 3 lag (172)
done with 6 lag (172)
done with 11 lag (172)
done with 22 lag (172)
done with 33 lag (172)
Done with Steve Novak in 8.0
done with 1 lag (567)
done with 2 lag (567)
done wit

In [162]:
with table.batch_writer() as batch:
    for row in lag.iterrows():
        batch.put_item(Item = row[1].to_dict())

In [181]:
dict_row = lag.iloc[0,:].to_dict()
key  = dict_row.pop('id')
ean = {"#"+key : key for key in dict_row.keys()}
eav = {":"+key : dict_row[key] for key in dict_row.keys()}
ue = "SET " + ', '.join(["#"+key +" = :"+key for key in dict_row.keys()])
with table.batch_writer() as batch:
    for row in lag.iterrows():
        dict_row = row[1].to_dict()
        key  = dict_row.pop('id')
        #ean = {"#"+key : key for key in dict_row.keys()}
        eav = {":"+key : dict_row[key] for key in dict_row.keys()}
        #ue = "SET " + ', '.join(["#"+key +" = :"+key for key in dict_row.keys()])
        table.update_item(Key={'id':key},
                 ExpressionAttributeNames = ean,
                 ExpressionAttributeValues = eav,
                 UpdateExpression = ue)

In [180]:
dict_row = lag.iloc[0,:].to_dict()
key  = dict_row.pop('id')
table.update_item(Key={'id':key},
                 ExpressionAttributeNames = ean,
                 ExpressionAttributeValues = eav,
                 UpdateExpression = ue)

ClientError: An error occurred (ValidationException) when calling the UpdateItem operation: Invalid UpdateExpression: An expression attribute value used in expression is not defined; attribute value: :w_06

In [172]:
ean = {"#"+key : key for key in dict_row.keys()}
eav = {":"+key : dict_row[key] for key in dict_row.keys()}
ue = "SET " + ', '.join(["#"+key +" = :"+key for key in dict_row.keys()])

In [175]:
key

'2016-01-01walljo01'

In [124]:
#lag['minutes_06'].apply(lambda x: Decimal(x))
pe = "slug, #dt, minutes_06"
fe = Key('date').lt('2016-11-07')
ea = {"#dt":"date"}
table.scan(ProjectionExpression = pe,ExpressionAttributeNames =ea, FilterExpression = fe)

{'Items': [{'slug': 'walljo01',
   'date': '2016-01-28',
   'minutes_06': Decimal('36.753')},
  {'slug': 'walljo01', 'date': '2016-10-27', 'minutes_06': Decimal('35.158')},
  {'slug': 'walljo01', 'date': '2016-02-06', 'minutes_06': Decimal('37.508')},
  {'slug': 'walljo01', 'date': '2016-01-11', 'minutes_06': Decimal('36.642')},
  {'slug': 'walljo01', 'date': '2016-10-30', 'minutes_06': Decimal('35.708')},
  {'slug': 'walljo01', 'date': '2016-02-19', 'minutes_06': Decimal('37.758')},
  {'slug': 'walljo01', 'date': '2016-01-26', 'minutes_06': Decimal('35.156')},
  {'slug': 'walljo01', 'date': '2016-02-29', 'minutes_06': Decimal('34.475')},
  {'slug': 'walljo01', 'date': '2016-03-12', 'minutes_06': Decimal('36.697')},
  {'slug': 'walljo01', 'date': '2016-01-08', 'minutes_06': Decimal('37.487')},
  {'slug': 'walljo01', 'date': '2016-01-01', 'minutes_06': Decimal('38.967')},
  {'slug': 'walljo01', 'date': '2016-01-09', 'minutes_06': Decimal('37.023')},
  {'slug': 'walljo01', 'date': '2016-

In [160]:
table.query(IndexName = 'slug-date-index', KeyConditionExpression = Key('slug').eq('walljo01') & Key('date').lt('2016-01-20'),
            ExpressionAttributeNames =ea,
            ProjectionExpression = pe,
            ScanIndexForward = False,
            Limit = 1
    )['Items'][0]

{'slug': 'walljo01', 'date': '2016-01-18', 'minutes_06': Decimal('36.369')}

In [141]:
table.get_item(IndexName = 'slug-date-index', KeyConditionExpression = Key('slug').eq('walljo01') & Key('date').lt('2016-01-20'),
            ExpressionAttributeNames =ea,
            ProjectionExpression = pe,
            ScanIndexForward = False,
               
    )

ParamValidationError: Parameter validation failed:
Missing required parameter in input: "Key"
Unknown parameter in input: "IndexName", must be one of: TableName, Key, AttributesToGet, ConsistentRead, ReturnConsumedCapacity, ProjectionExpression, ExpressionAttributeNames
Unknown parameter in input: "KeyConditionExpression", must be one of: TableName, Key, AttributesToGet, ConsistentRead, ReturnConsumedCapacity, ProjectionExpression, ExpressionAttributeNames
Unknown parameter in input: "ScanIndexForward", must be one of: TableName, Key, AttributesToGet, ConsistentRead, ReturnConsumedCapacity, ProjectionExpression, ExpressionAttributeNames

In [82]:
lag.head()

,slug,date,minutes_06,dk_score_06,dk_per_min_06,dk_no_fg_06,dk_no_fg_per_min_06,assists_06,assists_per_min_06,blocks_06,...,turnovers_per_min_06,doubles_06,fg_pct_06,ft_pct_06,two_pct_06,three_pct_06,game_score_06,attempted_field_goals_06,attempted_three_point_field_goals_06,w_06
0,walljo01,1/1/2016,38.966667,51.000000,1.308811,27.000000,0.692900,13.000000,0.333618,1.0,...,0.128315,2.0,0.526316,1.000000,0.625000,0.000000,22.100000,19.000000,3.000000,1.000000
1,walljo01,1/10/2017,39.566667,54.500000,1.376397,29.250000,0.738566,13.500000,0.341083,1.0,...,0.126398,2.0,0.525063,0.875000,0.575658,0.250000,22.450000,20.000000,2.500000,1.000000
2,walljo01,1/10/2018,40.488889,58.333333,1.437283,29.333333,0.724661,12.666667,0.314003,1.0,...,0.147258,2.0,0.522881,0.773810,0.558375,0.333333,23.666667,22.333333,3.666667,0.666667
3,walljo01,1/11/2016,39.050000,55.687500,1.421652,29.437500,0.757627,12.000000,0.307479,1.0,...,0.124839,2.0,0.484266,0.830357,0.502115,0.375000,21.625000,21.500000,3.750000,0.750000
4,walljo01,1/11/2017,38.983333,52.900000,1.352991,30.100000,0.775280,11.800000,0.302806,1.0,...,0.105037,1.8,0.425508,0.764286,0.451692,0.300000,18.980000,21.400000,4.000000,0.600000


In [14]:
data.sort_values(by = 'date',inplace=True)

In [ ]:
alpha = 0.3
n = 561
f2 = 'dk_score'
combos = [(('dk_per_k','median'),('dk_per_k','mean')),
          (('dk_per_k','median'),('dk_score','median')),
          (('dk_per_k','median'),('salary','mean')),
          (('salary','mean'),('dk_per_k','mean')),
          #('salary','dk_per_k'),
          #('minutes','dk_per_min'),
          #('salary','dk_score'),
          #('salary','minutes')
          #('salary','dk_per_min'),
          #('dk_score','dk_per_min')
         ]
fig, ax = plt.subplots(len(combos),1,figsize=(24,36))
buckets = [get_n_scatter(summary,elem[0],elem[1],ax = ax[ind],alpha= alpha,n = n, s = 44) for ind,elem in enumerate(combos)]
#buckets = get_n_scatter(data,f1,f2,ax = ax[0], alpha , n )
plt.show()